In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

En la actualidad, las organizaciones que contratan científicos de datos están especialmente interesadas en la cartera de candidatos para trabajos. El análisis de los datos de marketing de la organización es una de las aplicaciones más típicas de la ciencia de datos y el aprendizaje automático. Tal análisis definitivamente será una buena contribución a la cartera.

En general, los conjuntos de datos que contienen datos de marketing se pueden usar para 2 objetivos comerciales diferentes:

Predicción de los resultados de la campaña de marketing para cada cliente y aclaración de los factores que afectan los resultados de la campaña. Esto ayuda a descubrir las formas de hacer que las campañas de marketing sean más eficientes.
Conocer segmentos de clientes, utilizando datos de clientes que se suscribieron al depósito a plazo. Esto ayuda a identificar el perfil de un cliente, que tiene más probabilidades de adquirir el producto y desarrollar campañas de marketing más específicas.

Este conjunto de datos contiene datos de campañas de marketing bancario y podemos usarlo para optimizar las campañas de marketing para atraer a más clientes a la suscripción de depósitos a plazo. La descripción detallada del contenido del conjunto de datos se describe en el siguiente data set de kaggle(https://www.kaggle.com/janiobachmann/marketing-in-banking-opening-term-deposits).

Para optimizar las campañas de marketing con la ayuda del conjunto de datos, tendremos que seguir los siguientes pasos:

Importe datos del conjunto de datos y realice un análisis de alto nivel inicial: observe el número de filas, observe los valores faltantes, observe las columnas del conjunto de datos y sus valores correspondientes al resultado de la campaña.
Limpie los datos: elimine columnas irrelevantes, trate con valores faltantes e incorrectos, convierta columnas categóricas en variables ficticias.
Use técnicas de aprendizaje automático para predecir el resultado de la campaña de marketing y para descubrir los factores que afectan el éxito de la campaña.

Introducción de marketing:
El proceso por el cual las empresas crean valor para los clientes y construyen relaciones sólidas con los clientes para capturar el valor de los clientes a cambio.

Kotler y Armstrong (2010).

Las campañas de marketing se caracterizan por centrarse en las necesidades del cliente y su satisfacción general. Sin embargo, existen diferentes variables que determinan si una campaña de marketing será exitosa o no. Hay ciertas variables que debemos tener en cuenta al hacer una campaña de marketing.

Las 4 P:
1) Segmento de la población: ¿A qué segmento de la población se dirigirá la campaña de marketing y por qué? Este aspecto de la campaña de marketing es extremadamente importante ya que le dirá a qué parte de la población probablemente debería recibir el mensaje de la campaña de marketing.

2) Canal de distribución para llegar al lugar del cliente: implementar la estrategia más efectiva para aprovechar al máximo esta campaña de marketing. ¿Qué segmento de la población deberíamos abordar? ¿Qué instrumento debemos usar para difundir nuestro mensaje? (Ej: teléfonos, radio, televisión, redes sociales, etc.)

3) Precio: ¿Cuál es el mejor precio para ofrecer a clientes potenciales? (En el caso de la campaña de marketing del banco, esto no es necesario ya que el interés principal del banco es que los clientes potenciales abran cuentas de depósito para que las actividades operativas del banco sigan funcionando).

4) Estrategia promocional: esta es la forma en que se implementará la estrategia y cómo se abordarán los clientes potenciales. Esta debería ser la última parte del análisis de la campaña de marketing, ya que tiene que haber un análisis en profundidad de las campañas anteriores (si es posible) para aprender de los errores anteriores y determinar cómo hacer que la campaña de marketing sea mucho más efectiva.

¿Qué es un depósito a plazo? (Pudiera ser un credito, pero no logre entender 
Un depósito a plazo es un depósito que ofrece un banco o una institución financiera con una tasa fija (a menudo mejor que solo abrir una cuenta de depósito) en la que se le devolverá su dinero en un momento específico de vencimiento. Para obtener más información con respecto a los depósitos a plazo, haga clic en este enlace de Investopedia: https://www.investopedia.com/terms/t/termdeposit.asp

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import os

In [ ]:
df = pd.read_csv('../input/bank-marketing-dataset/bank.csv')

In [ ]:
df.head()

# **Exploración de datos**

In [ ]:
#número de filas en el conjunto de datos
print("Bank marketing dataset consists of {rows} rows.".format(rows = len(df)))

In [ ]:
#encontrar el porcentaje de valores faltantes para cada columna
missing_values = df.isnull().mean()*100

missing_values.sum()

Respuesta: Vemos que no faltan valores.

# **Exploración de columnas categóricas **

> En el conjunto de datos tenemos columnas categóricas y numéricas. Vamos primero a conocer los valores de las columnas categóricas.

In [ ]:
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month','poutcome']

fig, axs = plt.subplots(3, 3, sharex=False, sharey=False, figsize=(20, 15))

counter = 0
for cat_column in cat_columns:
    value_counts = df[cat_column].value_counts()
    
    trace_x = counter // 3
    trace_y = counter % 3
    x_pos = np.arange(0, len(value_counts))
    
    axs[trace_x, trace_y].bar(x_pos, value_counts.values, tick_label = value_counts.index)
    
    axs[trace_x, trace_y].set_title(cat_column)
    
    for tick in axs[trace_x, trace_y].get_xticklabels():
        tick.set_rotation(90)
    
    counter += 1

plt.show()

# *# **Exploración de columnas numéricas***

Mostraremos los valores de las columnas numéricas en histogramas. 

In [ ]:
num_columns = ['balance', 'day','duration', 'campaign', 'pdays', 'previous']

fig, axs = plt.subplots(2, 3, sharex=False, sharey=False, figsize=(20, 15))

counter = 0
for num_column in num_columns:
    
    trace_x = counter // 3
    trace_y = counter % 3
    
    axs[trace_x, trace_y].hist(df[num_column])
    
    axs[trace_x, trace_y].set_title(num_column)
    
    counter += 1

plt.show()

Podemos ver que las columnas numéricas tienen valores atípicos (especialmente 'pdays', 'campaign' y 'previous'). Posiblemente haya valores incorrectos, por lo que analizaremos de cerca los datos y decidir cómo manejamos para que noa efecte en el análisis. Veamos más de cerca los valores de las columnas 'campaign', 'pdays' y 'previous':

In [ ]:
df[['pdays', 'campaign', 'previous']].describe()

In [ ]:
len (df[df['pdays'] > 400] ) / len(df) * 100

'pdays' contiene la cantidad de días que pasaron después de que el cliente fue contactado por última vez desde una campaña anterior
Mirando más de cerca los datos de 'pdays' podemos ver que:
* solo el 1.2% de los valores superiores a 400. Posiblemente son valores atípicos, por lo que deberíamos considerar imputar algo (posiblemente un valor medio) en lugar de estos valores.
* -1 posiblemente significa que el cliente no fue contactado antes o representa datos faltantes.

Como no estamos seguros exactamente de lo que significa -1, sugiero eliminar esta columna, porque -1 representa más del 50% de los valores de la columna.

In [ ]:
len (df[df['campaign'] > 34] ) / len(df) * 100

'campaign'contiene el número de contactos realizados durante esta campaña y para este cliente (numérico, incluye el último contacto) Los números para 'campaña' por encima de 34 son claramente extraños, por lo que vamos a imputarlos con valores promedio de campaña mientras se limpian los datos.

In [ ]:
len (df[df['previous'] > 34] ) / len(df) * 100

Los números para 'previous' por encima de 34 también son realmente extraños, por lo que debemos imputarlos con valores promedio de campaña mientras se limpian los datos.

Es muy importante mirar la columna de respuesta, que contiene la información que vamos a predecir. En nuestro caso, es por eso que le pondremos mayor atención a la columna 'depósit' y comparar sus valores con otras columnas.

In [ ]:
value_counts = df['deposit'].value_counts()

value_counts.plot.bar(title = 'Valores de Deposit y counts')

En el diagrama vemos que los valores de "yes" y "no" para "depósit" están cerca, por lo que podemos usar la precisión como una métrica para un modelo, que predice el resultado de la campaña.

Analizaremos el valor de la columna 'deposit' dependiendo de los valores de otras columnas categóricas:

In [ ]:
#marital status y deposit

j_df = pd.DataFrame()

j_df['yes'] = df[df['deposit'] == 'yes']['job'].value_counts()
j_df['no'] = df[df['deposit'] == 'no']['job'].value_counts()

j_df.plot.bar(title = 'Job y deposit')

In [ ]:
#marital status y deposit
j_df = pd.DataFrame()

j_df['yes'] = df[df['deposit'] == 'yes']['marital'].value_counts()
j_df['no'] = df[df['deposit'] == 'no']['marital'].value_counts()

j_df.plot.bar(title = 'Marital status y deposit')

In [ ]:
#education y deposit
j_df = pd.DataFrame()

j_df['yes'] = df[df['deposit'] == 'yes']['education'].value_counts()
j_df['no'] = df[df['deposit'] == 'no']['education'].value_counts()

j_df.plot.bar(title = 'Education y deposit')

In [ ]:
#type of contact y deposit
j_df = pd.DataFrame()

j_df['yes'] = df[df['deposit'] == 'yes']['contact'].value_counts()
j_df['no'] = df[df['deposit'] == 'no']['contact'].value_counts()

j_df.plot.bar(title = 'Type of contact y deposit')

Con respecto a los diagramas, podemos decir que de acuerdo con nuestro conjunto de datos:

Los clientes con trabajos de 'blue-collar' y 'services' tienen menos probabilidades de suscribirse.
Los clientes casados ​​tienen menos probabilidades de suscribirse.
Los clientes con tipo de contacto 'cellular' tienen menos probabilidades de suscribirse.

Ahora vamos a analizar cómo las columnas numéricas afectan la suscripción de depósito a plazo.

In [ ]:
#balance y deposit

b_df = pd.DataFrame()
b_df['balance_yes'] = (df[df['deposit'] == 'yes'][['deposit','balance']].describe())['balance']
b_df['balance_no'] = (df[df['deposit'] == 'no'][['deposit','balance']].describe())['balance']

b_df

In [ ]:
b_df.drop(['count', '25%', '50%', '75%']).plot.bar(title = 'Estadisticas de Balance y deposit')

In [ ]:
c_df = pd.DataFrame()
c_df['campaign_yes'] = (df[df['deposit'] == 'yes'][['deposit','campaign']].describe())['campaign']
c_df['campaign_no'] = (df[df['deposit'] == 'no'][['deposit','campaign']].describe())['campaign']

c_df

In [ ]:
c_df.drop(['count', '25%', '50%', '75%']).plot.bar(title = 'Número de contactos realizados durante esta campaña y estadísticas de depósito')

In [ ]:
p_df = pd.DataFrame()
p_df['previous_yes'] = (df[df['deposit'] == 'yes'][['deposit','previous']].describe())['previous']
p_df['previous_no'] = (df[df['deposit'] == 'no'][['deposit','previous']].describe())['previous']

p_df

In [ ]:
p_df.drop(['count', '25%', '50%', '75%']).plot.bar(title = 'Número de contactos realizados durante campañas anteriores y estadísticas de depósito')

**Mirando los diagramas anteriores, podemos concluir que:

Las personas que se suscribieron al depósito a plazo tienden a tener un mayor saldo y valores de edad.
Las personas que se suscribieron al depósito a plazo tienden a tener menos contactos durante esta campaña.

## Limpieza de datos

Convertimos columnas con valores 'yes' y 'no' en columnas booleanas;
Convertimos columnas categóricas en dummy

In [ ]:
def get_dummy_from_bool(row, column_name):
    ''' Returns 0 if value in column_name is no, returns 1 if value in column_name is yes'''
    return 1 if row[column_name] == 'yes' else 0

def get_correct_values(row, column_name, threshold, df):
    ''' Returns mean value if value in column_name is above threshold'''
    if row[column_name] <= threshold:
        return row[column_name]
    else:
        mean = df[df[column_name] <= threshold][column_name].mean()
        return mean

def clean_data(df):
    
    cleaned_df = df.copy()
    
    bool_columns = ['default', 'housing', 'loan', 'deposit']
    for bool_col in bool_columns:
        cleaned_df[bool_col + '_bool'] = df.apply(lambda row: get_dummy_from_bool(row, bool_col),axis=1)
    
    cleaned_df = cleaned_df.drop(columns = bool_columns)
    
 
    cat_columns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
    
    for col in  cat_columns:
        cleaned_df = pd.concat([cleaned_df.drop(col, axis=1),
                                pd.get_dummies(cleaned_df[col], prefix=col, prefix_sep='_',
                                               drop_first=True, dummy_na=False)], axis=1)
    
   
    cleaned_df = cleaned_df.drop(columns = ['pdays'])
    
    
    cleaned_df['campaign_cleaned'] = df.apply(lambda row: get_correct_values(row, 'campaign', 34, cleaned_df),axis=1)
    cleaned_df['previous_cleaned'] = df.apply(lambda row: get_correct_values(row, 'previous', 34, cleaned_df),axis=1)
    
    cleaned_df = cleaned_df.drop(columns = ['campaign', 'previous'])
    
    return cleaned_df

In [ ]:
#limpieza del conjunto de datos
cleaned_df = clean_data(df)
cleaned_df.head()

## Machine Learning para la predicción del resultado de la campaña.

In [ ]:
### Usaremos el modelo de clasificación para la predicción del resultado de la campaña.

Ahora usemos conjuntos de datos limpios para predecir el resultado de la campaña con la ayuda de modelos de clasificación de aprendizaje automático. Usaré XGBoost, que es una de las bibliotecas de aprendizaje automático más comunes para modelar. El modelo resultante también me ayudará a comprender qué características tienen la mayor importancia para la predicción de los resultados de la campaña.

Creeamos un conjuntos de datos X y para entrenar el modelo dividimos el conjuntos de datos de entrenamiento y prueba

In [ ]:
X = cleaned_df.drop(columns = 'deposit_bool')
y = cleaned_df[['deposit_bool']]

In [ ]:
TEST_SIZE = 0.3
RAND_STATE = 42

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state=RAND_STATE)

In [ ]:
#Train XGBoost classifier model:

In [ ]:
#train XGBoost model
xgb = xgboost.XGBClassifier(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(X_train,y_train.squeeze().values)


y_train_preds = xgb.predict(X_train)
y_test_preds = xgb.predict(X_test)

print('XGB accuracy score for train: %.3f: test: %.3f' % (
        accuracy_score(y_train, y_train_preds),
        accuracy_score(y_test, y_test_preds)))

Obtenemos las características importantes del modelo entrenado:

In [ ]:
headers = ["name", "score"]
values = sorted(zip(X_train.columns, xgb.feature_importances_), key=lambda x: x[1] * -1)
xgb_feature_importances = pd.DataFrame(values, columns = headers)


x_pos = np.arange(0, len(xgb_feature_importances))
plt.bar(x_pos, xgb_feature_importances['score'])
plt.xticks(x_pos, xgb_feature_importances['name'])
plt.xticks(rotation=90)
plt.title('Feature importances (XGB)')

plt.show()

Estás son las caractetisticas más importantes

* Saldo de la cuenta del cliente,
* Edad del cliente,
* Número de contactos realizados durante esta campaña y duración del contacto,
* Número de contactos realizados antes de esta campaña.

Entonces, los principales resultados del modelado son:

* Los clientes de mayor edad tienen más probabilidades de suscribirse para el depósit a plazo.
* Los clientes con mayor saldo de cuenta tienen más probabilidades de suscribirse para el depósit a plazo.
* El número de contactos con los clientes realmente importa. Demasiados contactos con el cliente podrían hacer que rechace la oferta.

# **Análisis Extra**

1. Averigüe el saldo de la cuenta, en qué campaña de marketing debe centrarse:

In [ ]:
df_new = cleaned_df.copy()


df_new['balance_buckets'] = pd.qcut(df_new['balance'], 50, labels=False, duplicates = 'drop')


mean_deposit = df_new.groupby(['balance_buckets'])['deposit_bool'].mean()


plt.plot(mean_deposit.index, mean_deposit.values)
plt.title('Mean % subscription depending on account balance')
plt.xlabel('balance bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['balance_buckets'] == 34]['balance'].min()

Del diagrama anterior podemos concluir que las campañas de marketing deben concentrarse en clientes con un saldo de cuenta mayor a $ 1490.

In [ ]:
df_new['age_buckets'] = pd.qcut(df_new['age'], 20, labels=False, duplicates = 'drop')

mean_age = df_new.groupby(['age_buckets'])['deposit_bool'].mean()


plt.plot(mean_age.index, mean_age.values)
plt.title('Porcentaje de suscripción dependiendo la edad')
plt.xlabel('age bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['age_buckets'] == 3]['age'].max()

In [ ]:
df_new[df_new['age_buckets'] == 17]['age'].min()

Por lo tanto, vemos que la tasa de suscripción promedio tiende a ser mayor para los clientes menores de 31 años o mayores de 56 años.

In [ ]:
Averigüamos la cantidad adecuada de contactos con el cliente durante la campaña:

In [ ]:
df_new['campaign_buckets'] = pd.qcut(df_new['campaign_cleaned'], 20, labels=False, duplicates = 'drop')

mean_campaign = df_new.groupby(['campaign_buckets'])['deposit_bool'].mean()


plt.plot(mean_campaign.index, mean_campaign.values)
plt.title('Porcentaje de suscripción dependiendo en número de contactos')
plt.xlabel('number of contacts bucket')
plt.ylabel('% subscription')
plt.show()

In [ ]:
df_new[df_new['campaign_buckets'] == 2]['campaign_cleaned'].min()

En la gráfica anterior, vemos que la tasa de suscripción promedio es inferior al 50% si el número de contactos durante la campaña supera los 4.

# **Conclusión**

Los resultados clave del análisis son las recomendaciones para futuras campañas de marketing:

* El saldo de la cuenta del cliente tiene una gran influencia en el resultado de la campaña. Las personas con un saldo de cuenta superior a $1490 tienen más probabilidades de suscribirse para un depósito a plazo, por lo que en el futuro se dirigirán a esos clientes.
* La edad del cliente también afecta el resultado de la campaña. Las campañas futuras deben concentrarse en clientes de categorías de edad menores de 30 años y mayores de 50 años.
* El número de contactos con el cliente durante la campaña también es muy importante. El número de contactos con el cliente no debe exceder 4.

# **Extra...**

Sería interesante saber cuál fue el mes con mayor actividad en Marketing, conocer cuál es el mes con mayores rechazos de las ofertas. El mes con menor actividad, mes con mayores contratos y suscripciones.

Conocer cuántas llamadas se les podría hacer a los clientes, y evitar que se molesten. 

Profundizar más en los rangos de edades para poder realizar campañas aún más profundas. Campañas personalizadas. 

Conocer más de sus demografía. 